In [1]:
import tensorflow as tf
import numpy as np

# Filt-Filt operation definition

- FeedBack filter components `[M]`:   $A = a_0, a_1, a_2 \dots a_M$, where $a_0 = 0$ by definition
- FeedForward filter components`[N]`:   $B = b_0, b_1, b_2 \dots b_N$

```
y(t) = x(t)*b(0) + x(t-1)*b(1) + x(t-2)*b(2) ... x(t-M)*b(N)
                 - y(t-1)*a(1) - y(t-2)*a(2) ... y(t-N)*a(M)
```

$$
y_t = \sum_{i=0}^{N} x_{t-i} \cdot b_{i} - \sum_{i=1}^{M} y_{t-i} \cdot a_{i}
$$


```
y(t) = x[t-N : t] * b[::-1] - y[t-M : t-1] * a[::-1]
```

for faster computation let $s$ be $s := x \ast b$, where $\ast$ is the convolution operator, since all values of $x$ is known at computation time

```
y(t) = h[t-N : t] - y[y-t : t-1] * a[::-1]
```


In [4]:
def resetSession():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    return sess

In [38]:
sess = resetSession()
# Input
x = tf.placeholder(tf.float32, shape=[None], name='x_vector')

# Feedback filter
a = tf.placeholder(tf.float32, shape=[None], name='a_vector')
# Feedforward filter
b = tf.placeholder(tf.float32, shape=[None], name='b_vector')

# Support vector
# Default convolution works like this
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  e  f    ]
#            +
#         result

# But this time we need the rightmost value as a result
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  e  f    ]
#                  +
#               result
#
# So we use padding like a Jedi, and use VALID convolution
x_padded = tf.pad(x, [[tf.shape(b)[0]-1, tf.shape(b)[0]-1]])

# value[None, :, None] stands for [Batch, Sample, Channels]
# In the basic example Batch = Channel = 1
# filters[::-1, None, None] stands for [Filter_size(inverted), In_size, Out_size]
# In the basic example In_size = Out_size = 1
s = tf.nn.conv1d(value=x_padded[None, :, None], filters=b[::-1, None, None], stride=1, padding='VALID')
s = tf.squeeze(s, name='s_vector')

In [48]:
x_test = list(range(1, 5))
b_test = [0, 1, 0]
s.eval({x:x_test, b:b_test})

array([ 0.,  1.,  2.,  3.,  4.,  0.], dtype=float32)

In [49]:
# Watch out, the filter is now inverted
b_test = [0, 0, 1]
s.eval({x:x_test, b:b_test})

array([ 0.,  0.,  1.,  2.,  3.,  4.], dtype=float32)

In [50]:
b_test = [1, 0, 0]
s.eval({x:x_test, b:b_test})

array([ 1.,  2.,  3.,  4.,  0.,  0.], dtype=float32)

In [ ]:
# Output
